In [1]:
from pyspark.sql import SparkSession
import os

# Включаем поддержку Hive (хотя в локальном докере это часто эмуляция через Derby)
spark = SparkSession.builder \
    .appName("Catalog_Lab") \
    .enableHiveSupport() \
    .getOrCreate()

# 1. Создаем базу данных
spark.sql("CREATE DATABASE IF NOT EXISTS retail_db")

# 2. Переключаемся на неё
spark.sql("USE retail_db")

# Проверим, где Spark собирается хранить таблицы этой базы
spark.sql("DESCRIBE DATABASE retail_db").show(truncate=False)

+--------------+--------------------------------------------------------------------------------------------+
|info_name     |info_value                                                                                  |
+--------------+--------------------------------------------------------------------------------------------+
|Catalog Name  |spark_catalog                                                                               |
|Namespace Name|retail_db                                                                                   |
|Comment       |                                                                                            |
|Location      |file:/home/jovyan/workspace/MODULES/MODULE_05/LESSON_03_CATALOG/spark-warehouse/retail_db.db|
|Owner         |jovyan                                                                                      |
+--------------+--------------------------------------------------------------------------------------------+



### Managed Table

In [2]:
# Создаем данные
df = spark.createDataFrame([(1, "Item A"), (2, "Item B")], ["id", "name"])

# Сохраняем как MANAGED таблицу (просто saveAsTable без указания пути)
df.write.mode("overwrite").saveAsTable("managed_items")

# Проверим, где она лежит
spark.sql("DESCRIBE EXTENDED managed_items").filter("col_name = 'Location'").show(truncate=False)
# Вы увидите путь внутри папки spark-warehouse

# Удаление (ВНИМАНИЕ: Удалит и файлы!)
# spark.sql("DROP TABLE managed_items")

+--------+----------------------------------------------------------------------------------------------------------+-------+
|col_name|data_type                                                                                                 |comment|
+--------+----------------------------------------------------------------------------------------------------------+-------+
|Location|file:/home/jovyan/workspace/MODULES/MODULE_05/LESSON_03_CATALOG/spark-warehouse/retail_db.db/managed_items|       |
+--------+----------------------------------------------------------------------------------------------------------+-------+



### External Table (Безопасно)

In [3]:
# 1. Сначала пишем файлы в произвольное место (например, /tmp/data)
external_path = "/home/jovyan/workspace/data/external_items"
df.write.mode("overwrite").parquet(external_path)

# 2. Создаем ВНЕШНЮЮ таблицу, указывая LOCATION
spark.sql(f"""
    CREATE TABLE external_items (
        id INT,
        name STRING
    )
    USING PARQUET
    LOCATION '{external_path}'
""")

# Проверим тип
spark.sql("DESCRIBE EXTENDED external_items").filter("col_name = 'Type'").show()
# Должно быть: EXTERNAL

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    Type| EXTERNAL|       |
+--------+---------+-------+



### Python API каталога

In [4]:
# Получить список таблиц в базе
tables = spark.catalog.listTables("retail_db")

for t in tables:
    print(f"Table: {t.name}, Type: {t.tableType}")

Table: external_items, Type: EXTERNAL
Table: managed_items, Type: MANAGED


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 45572)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

### Анти-паттерны и Ошибки

🚫 Ошибка 1: CTAS (Create Table As Select) без понимания типа
- Код: df.write.saveAsTable("my_backup")
- Проблема: Это создает Managed таблицу. Если вы или коллега решите "почистить базу" и сделаете DROP TABLE my_backup, бэкап исчезнет физически.
- Решение: Всегда явно используйте .option("path", "/secure/location") при saveAsTable, чтобы сделать её внешней (в некоторых версиях) или создавайте через SQL CREATE EXTERNAL TABLE.

🚫 Ошибка 2: Рассинхрон Метаданных  
- Сценарий: Вы удалили файлы руками через терминал (rm -rf ...), но не удалили таблицу в Spark.
- Результат: Spark думает, что таблица есть, но при SELECT падает с ошибкой FileNotFoundException.
- Решение: Команда MSCK REPAIR TABLE my_table (для партиционированных таблиц) или REFRESH TABLE my_table.